# TM10007 Assignment template

In [124]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/mamolenaar/TM10007_PROJECT_SvdMeijden_MMolenaar.git      #SIRI: mamolenaar moet je even aanpassen

##Taakverdeling eindopdracht:


---



1.   **Datapreperatie**


* *Verwijderen duplicaten kolommen (als aanwezig)*   DONE 
*  *Nulwaarden verwijderen* (Siri)        DONE
* *Trainingset/validatieset* (Mitch)       DONE
* Testing outliers with Z-score (Mitch)
*   Scaling features (Mitch)
* PCA?
* Feature selection 


2.   **Random Forest**
* Bagging/bootstrap aggregating 
* Hyperparameter tuning
* Ensembling 
3. **Support Vector Machine**
* Kernel
* Hyperparameter tuning 
4. **Multilayer neural network** 
* Activation and loss function
5. **Convolutional NN/Recurrent NN** 
(Ik weet niet zeker of we deze ook moeten toepassen) 


Checklist punten verslag / punten om overna te denken
1. Datapreparatie
- Hoeveel nulwaarden/NaNs zijn er? in welke kolommen komen deze voor? 
  komen er veel NaNs voor in dezelfde kolom? Zo ja, is het geoorloofd om deze kolommen te imputeren? Op welke manier imputeren (op basis van andere features (multivariate imputation) of eigen features: zie https://scikit-learn.org/stable/modules/impute.html

- Imputatie: https://towardsdatascience.com/handling-missing-data-for-a-beginner-6d6f5ea53436

- Hebben we veel kolommen waarin outliers zitten? Zo ja, dan moeten we hierop scaling aanpassen, zie: https://scikit-learn.org/stable/modules/preprocessing.html. Testen outliers met Z-score:https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba

- We nemen aan dat basic classifiers (KNN, linear, quadratic etc.) te simpel zijn om te gebruiken (toch?). Wat kan het doen van feature selection hierover zeggen?
Als we toch basic classifiers gebruiken, moeten we hier voorafgaand PCA op toepassen?


In [0]:
######## Import packages and models ##########################################

# Import general packages
import pandas as pd
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
import seaborn
from sklearn import model_selection


# import classifiers
from sklearn import model_selection
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# import models
models = [
    ('NB', GaussianNB()),
    ('LDA', LinearDiscriminantAnalysis()),
    ('QDA', QuadraticDiscriminantAnalysis()),
    ('LR', LogisticRegression()),
    ('SGD', SGDClassifier()),
    ('DT', DecisionTreeClassifier()),
    ('KNN', KNeighborsClassifier()),   #hieronder verdere modellen toevoegen
]

# Import datapreprocessing
from sklearn.impute import SimpleImputer

# Import split data in train and test set
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score # allows one scoring metric
from sklearn.model_selection import cross_validate # allows us to use multiple scoring metrics

# Import metrics
from sklearn import metrics

In [126]:
# Data loading functions. 
from hn.load_data import load_data
data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
data.info
# Find labels assigned for tumor classification 
def unique(list1): 
    x = np.array(list1) 
    print(np.unique(x)) 

print("The unique labels assigned to each patient for tumor classification are") 
unique(data['label'])


The number of samples: 113
The number of columns: 160
The unique labels assigned to each patient for tumor classification are
['T12' 'T34']


In [127]:
########################## Data preperation ##########################

# Replace zero values with NaN 
data = data.replace(0, np.NaN)
# count the number of NaN values in each column
print(data.isnull().sum())

data = data.loc[:,~data.columns.duplicated()]   # delete duplicated columns if available
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean') # impute NaN values with mean values


hf_energy               0
hf_entropy              0
hf_kurtosis             0
hf_max                  0
hf_mean                 0
                       ..
tf_NGTDM_Busyness      54
tf_NGTDM_Coarseness     0
tf_NGTDM_Complexity    54
tf_NGTDM_Contrast      54
tf_NGTDM_Strength      54
Length: 160, dtype: int64
               hf_energy  hf_entropy  ...  tf_NGTDM_Contrast  tf_NGTDM_Strength
ID                                    ...                                      
0_HN1006_0  24345.357124    3.444203  ...                NaN                NaN
0_HN1022_0  35301.370880    2.873434  ...       1.090935e-04           0.018735
0_HN1026_0   6340.950214    2.769541  ...       3.528037e-06           0.672940
0_HN1029_0   5690.500179    3.073200  ...       1.917977e-04           0.062940
0_HN1046_0  15553.548185    3.025631  ...       2.093031e-04           0.030421
...                  ...         ...  ...                ...                ...
0_HN1950_0  50004.678306    2.246432  ...       

In [128]:
##########################Comparing models ###########################
## Hieronder eerst splitsing in train (validatie en train) en test set. V
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # split train and test
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) # split train in validation and train

# Hieronder toepassen van de verschillende modellen op de trainingsdata. Vervolgens berekenen metrics op validatie set. 
# Bereking metrics op testset is nog niet gebeurt. Hiervoor moet nog optimalisatie van parameters plaatsvinden (hoeveel splitsingen zijn nodig etc.)
dic= {}
scoring = ['roc_auc', 'accuracy', 'f1_macro', 'precision_macro', 'recall_macro']
for name, model in models:
        clf = model
        clf.fit(X_train, y_train)
        accuracy = clf.score(X_val, y_val)        # accuracy tested on validation data without cross validation
        #accuracy_CV = cross_val_score(clf, X, y, cv=5)
        for metric in scoring:                      # cross validation
            accuracy_CV = cross_validate(clf, X_val, y_val, cv=5, scoring=metric, return_train_score=False) # cv can also return train score but we set it to false
            test_score = accuracy_CV["test_score"]
            #dic = {name: test_score}
            dic[name, metric]=pd.DataFrame({'mean_score': {0:test_score.mean()}, 'std_score': {0:test_score.std()*2}})
##########################Cross- validation###########################

ValueError: ignored